<a href="https://colab.research.google.com/github/HazemmoAlsady/Search-Engine/blob/main/Search_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setup**

In [1]:
!pip install nltk PyPDF2

import nltk
nltk.download('stopwords')
nltk.download('wordnet')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# **Preprocessing**

In [2]:
import re
import math
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import PyPDF2



arabic_stopwords = set([
    "في", "على", "من", "إلى", "عن", "مع", "هذا", "هذه",
    "هو", "هي", "هم", "هن", "كان", "كانت", "يكون",
    "ما", "لا", "لم", "لن", "أن", "إن", "كل", "أي"
])

def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "و", text)
    text = re.sub("ئ", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("[ًٌٍَُِّْ]", "", text)  # tashkeel
    return text



In [3]:
stop_words_en = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower()
    text = normalize_arabic(text)
    text = re.sub(r"[^\w\s]", "", text)

    tokens = text.split()
    clean_tokens = []

    for t in tokens:
        # Arabic
        if re.match(r"[\u0600-\u06FF]+", t):
            if t not in arabic_stopwords:
                clean_tokens.append(t)

        # English
        elif t.isalpha():
            if t not in stop_words_en:
                clean_tokens.append(lemmatizer.lemmatize(t))

    return clean_tokens


# **Input Documents**

In [4]:
from google.colab import files
uploaded = files.upload()

print("Uploaded files:")
for f in uploaded.keys():
    print("-", f)


Saving Data Analysis Roadmap 2024.pdf to Data Analysis Roadmap 2024.pdf
Uploaded files:
- Data Analysis Roadmap 2024.pdf


# **Load TXT Documents**

In [5]:
def load_txt(filename):
    with open(filename, "r", encoding="utf-8") as f:
        return [line.strip() for line in f if line.strip()]


# **Load PDF Documents**

In [6]:
def load_pdf(filename):
    docs = []
    with open(filename, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text = page.extract_text()
            if text:
                paragraphs = text.split("\n")
                for p in paragraphs:
                    p = p.strip()
                    if len(p) > 20:
                        docs.append(p)
    return docs


In [7]:
docs = []

for filename in uploaded.keys():
    if filename.endswith(".txt"):
        docs.extend(load_txt(filename))
    elif filename.endswith(".pdf"):
        docs.extend(load_pdf(filename))

print(f"\nTotal documents loaded: {len(docs)} ✅")



Total documents loaded: 40 ✅


# **Build TF + IDF**

In [8]:
def build_tf(docs):
    tf_docs = []
    for doc in docs:
        freq = defaultdict(int)
        for token in preprocess(doc):
            freq[token] += 1
        tf_docs.append(freq)
    return tf_docs

tf_docs = build_tf(docs)


# **TF-IDF Vector + Cosine Similarity**

In [9]:
def compute_idf(tf_docs):
    N = len(tf_docs)
    df = defaultdict(int)

    for doc in tf_docs:
        for term in doc:
            df[term] += 1

    return {term: math.log(N / df[term]) for term in df}

idf = compute_idf(tf_docs)


In [10]:
print("Indexed terms:")
print(list(idf.keys())[:30])


Indexed terms:
['data', 'analysis', 'roadmap', 'ملحوظه', 'مهمه', 'جدا', 'لست', 'مطالب', 'بمعرفه', 'دوال', 'االكسيل', 'لكن', 'سنقوم', 'بسرد', 'المهم', 'ليك', 'ك', 'محلل', 'بيانات', 'وبعد', 'ذلك', 'يمكنك', 'ان', 'تزيد', 'علي', 'حسب', 'الحاجه', 'vlookup', 'hlookup', 'xlookup']


In [11]:
def tfidf_vector(tf_doc, idf):
    return {
        term: (1 + math.log(freq)) * idf.get(term, 0)
        for term, freq in tf_doc.items()
    }


In [12]:
def cosine(v1, v2):
    dot = sum(v1.get(t, 0) * v2.get(t, 0) for t in v1)
    norm1 = math.sqrt(sum(v**2 for v in v1.values()))
    norm2 = math.sqrt(sum(v**2 for v in v2.values()))
    if norm1 == 0 or norm2 == 0:
        return 0
    return dot / (norm1 * norm2)


# **Ranked Search Engine**

In [13]:
def ranked_search(query, docs, tf_docs, idf, top_k=5):
    q_tf = defaultdict(int)
    for token in preprocess(query):
        q_tf[token] += 1

    q_vec = tfidf_vector(q_tf, idf)

    scores = []
    for i, tf_doc in enumerate(tf_docs):
        d_vec = tfidf_vector(tf_doc, idf)
        score = cosine(q_vec, d_vec)
        scores.append((i, score))

    scores.sort(key=lambda x: x[1], reverse=True)
    return [(i, docs[i], score) for i, score in scores[:top_k] if score > 0]


# **Interactive Search Loop**

In [14]:
print("\nSearch Engine Ready 🔍 (TXT + PDF | Arabic + English)")
print("Type 'quit' to exit")

while True:
    query = input("\nSearch: ").strip()

    if not query:
        print("Please enter a query ❗")
        continue

    if query.lower() == "quit":
        print("Search Engine stopped ✅")
        break

    results = ranked_search(query, docs, tf_docs, idf)

    if not results:
        print("No results found ❌")
        continue

    for i, text, score in results:
        print(f"\nScore: {score:.3f}")
        print(text[:300])



Search Engine Ready 🔍 (TXT + PDF | Arabic + English)
Type 'quit' to exit

Search: بيانات

Score: 0.277
المهم ليك ك محلل بيانات وبعد ذلك يمكنك ان تزيد على حسب الحاجه

Search: excel
No results found ❌

Search: Excel
No results found ❌

Search: functions
No results found ❌

Search: Functions
No results found ❌

Search: Function
No results found ❌

Search: المختصر
No results found ❌

Search: مختصر
No results found ❌

Search: data

Score: 0.654
Data Visualization: -

Score: 0.466
Power BI Data Model: -

Score: 0.445
Data Analysis Roadmap 2024

Search: sql
No results found ❌

Search: website

Score: 0.577
4-  TEST & Interview on websites

Search: power

Score: 0.707
2- How to be power bi

Score: 0.564
Power BI Power Query (Advanced): -

Score: 0.520
Power BI Visualization: -

Score: 0.414
Power BI Data Model: -

Search: SQL
No results found ❌

Search: sql
No results found ❌

Search: test

Score: 0.577
4-  TEST & Interview on websites

Search: quit
Search Engine stopped ✅
